# Código para detectar codificación del txt

In [ ]:
import chardet  # chardet helps to detect the character encoding

def read_file_with_unknown_encoding(filename):
    try:
        # Use chardet to detect the encoding
        with open(filename, 'rb') as f:
            rawdata = f.read()
            result = chardet.detect(rawdata)
            encoding = result['encoding']

        # Now, read the file with the detected encoding
        with open(filename, 'r', encoding=encoding) as file:
            content = file.read()
            print(f"Successfully read with encoding: {encoding}")
            return content

    except UnicodeDecodeError:
        print(f"Failed with encoding: {encoding}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return None  # appropriate error handling based on your application's logic.



RX sense codec


In [ ]:
"""
Simple example of using the RF24 class.
"""
import time
import struct
import board
import os as sys
import subprocess
import glob
from digitalio import DigitalInOut


def fragmentFile(string, length):
    return list(string[0+i: length+i] for i in range(0, len(string), length))
def getUSBpath():
	rpistr = "/media/mtp/"
	proc = subprocess.Popen("ls "+rpistr,shell=True, preexec_fn=sys.setsid, stdout=subprocess.PIPE)
	line = proc.stdout.readline()
	print(str(line.rstrip()))
	path = rpistr + line.rstrip()+.decode("utf-8") +"/"
	return path

def openFile(path):
	file = open(glob.glob(path+'*.txt')[0],"rb")
	strF= file.read()
	return strF

def writeFile(path, buff):
	file = open(path+"result.txt","wb")
	file.write(buff)
	file.close()


# if running this on a ATSAMD21 M0 based board
# from circuitpython_nrf24l01.rf24_lite import RF24
from circuitpython_nrf24l01.rf24 import RF24
# invalid default values for scoping
SPI_BUS, CSN_PIN, CE_PIN = (None, None, None)

try:  # on Linux
    import spidev

    SPI_BUS = spidev.SpiDev()  # for a faster interface on linux
    CSN_PIN = DigitalInOut(board.D17) # use CE0 on default bus (even faster than using any pin)
    CE_PIN = DigitalInOut(board.D22)  # using pin gpio22 (BCM numbering)

except ImportError:  # on CircuitPython only
    # using board.SPI() automatically selects the MCU's
    # available SPI pins, board.SCK, board.MOSI, board.MISO
    SPI_BUS = board.SPI()  # init spi bus object

    # change these (digital output) pins accordingly
    CE_PIN = DigitalInOut(board.D4)
    CSN_PIN = DigitalInOut(board.D5)


# initialize the nRF24L01 on the spi bus object
nrf = RF24(SPI_BUS, CSN_PIN, CE_PIN)
# On Linux, csn value is a bit coded
#                 0 = bus 0, CE0  # SPI bus 0 is enabled by default
#                10 = bus 1, CE0  # enable SPI bus 2 prior to running this
#                21 = bus 2, CE1  # enable SPI bus 1 prior to running this

# set the Power Amplifier level to -12 dBm since this test example is
# usually run with nRF24L01 transceivers in close proximity
nrf.pa_level = -18
nrf.data_rate = 1

# addresses needs to be in a buffer protocol object (bytearray)
address = [b"1Node", b"2Node"]

# to use different addresses on a pair of radios, we need a variable to
# uniquely identify which address this radio will use to transmit
# 0 uses address[0] to transmit, 1 uses address[1] to transmit
nrf.print_details()
radio_number = bool(
    int(input("Which radio is this? Enter '0' or '1'. Defaults to '0' ") or 0)
)

# set TX address of RX node into the TX pipe
nrf.open_tx_pipe(address[radio_number])  # always uses pipe 0

# set RX address of TX node into an RX pipe
nrf.open_rx_pipe(1, address[not radio_number])  # using pipe 1

# using the python keyword global is bad practice. Instead we'll use a 1 item
# list to store our float number for the payloads sent

# uncomment the following 3 lines for compatibility with TMRh20 library
# nrf.allow_ask_no_ack = False
# nrf.dynamic_payloads = False
# nrf.payload_length = 4


def master():  # count = 5 will only transmit 5 packets
    """Transmits an incrementing integer every second"""
    payload_size = 32
    pth = getUSBpath()
    strF = openFile(pth)
    payload = fragmentFile(strF,payload_size)
    nrf.listen = False  # ensures the nRF24L01 is in TX mode
    count=len(payload)
    zero_timer = time.monotonic_ns()
    result = False
#     print(nrf.is_lna_enabled())
    for i in range(count):
        # use struct.pack to structure your data
        # into a usable payload
        limit = 100
        print(type(payload[i]))
        print(payload[i])
        buffer = payload[i]
        # "<f" means a single little endian (4 byte) float value.
        #start_timer = time.monotonic_ns()  # start timer
        result = nrf.send(buffer, False, 10)

        while not result and  limit:
            result = nrf.send(buffer, False, 10)
            time.sleep(0.5)
            limit -= 1
        end_timer = time.monotonic_ns()  # end timer

        if not result:
            print("send() failed or timed out")
        else:
            print(
                "Transmission successful! Time to Transmit:",
                "{} us. Sent: {}".format((end_timer - start_timer) / 1000, payload[i]),
            )
        print("Transmission rate: ", (((len(payload)*52)*8)/((end_timer-zero_timer)/1e9)))




def slave(timeout=10):
    """Polls the radio and prints the received value. This method expires
    after 6 seconds of no received transmission"""
    nrf.listen = True  # put radio into RX mode and power up

    start = time.monotonic()
    while (time.monotonic() - start) < timeout:
        if nrf.available():
            # grab information about the received payload
            payload_size, pipe_number = (nrf.any(), nrf.pipe)
            # fetch 1 payload from RX FIFO
            buffer = nrf.read()  # also clears nrf.irq_dr status flag
            # expecting a little endian float, thus the format string "<f"
            # buffer[:4] truncates padded 0s if dynamic payloads are disabled
            msg += buffer#.decode("utf-8")
            # print details about the received packet
            print(
                "Received {} bytes on pipe {}: {}".format(
                    payload_size, pipe_number, msg
                )
            )
            start = time.monotonic()

    # recommended behavior is to keep in TX mode while idle
    nrf.listen = False  # put the nRF24L01 is in TX mode
    path = "/home/mtp/MTP/"
    writeFile(path, msg)

def set_role():
    """Set the role using stdin stream. Timeout arg for slave() can be
    specified using a space delimiter (e.g. 'R 10' calls `slave(10)`)
    """
    user_input = (
        input(
            "*** Enter 'R' for receiver role.\n"
            "*** Enter 'T' for transmitter role.\n"
            "*** Enter 'Q' to quit example.\n"
        )
        or "?"
    )
    user_input = user_input.split()
    if user_input[0].upper().startswith("R"):
        slave(*[int(x) for x in user_input[1:2]])
        return True
    if user_input[0].upper().startswith("T"):
        master(*[int(x) for x in user_input[1:2]])
        return True
    if user_input[0].upper().startswith("Q"):
        nrf.power = False
        return False
    print(user_input[0], "is an unrecognized input. Please try again.")
    return set_role()


print("    nRF24L01 Simple test")

if __name__ == "__main__":
    try:
        while set_role():
            pass  # continue example until 'Q' is entered
    except KeyboardInterrupt:
        print(" Keyboard Interrupt detected. Powering down radio...")
        nrf.power = False
else:
    print("    Run slave() on receiver\n    Run master() on transmitter")

prueba con utf8

In [ ]:
# Use the function
file_content = read_file_with_unknown_encoding('/content/drive/MyDrive/codif/file_utf8.txt')
if file_content is not None:
    print(file_content)

Successfully read with encoding: utf-8
EnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcI

prueba con utf16

In [ ]:

# Use the function
file_content = read_file_with_unknown_encoding('/content/drive/MyDrive/codif/file_utf16.txt')
if file_content is not None:
    print(file_content)

Successfully read with encoding: UTF-16
EnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarc

prueba con utf32

In [ ]:
# Use the function
file_content = read_file_with_unknown_encoding('/content/drive/MyDrive/codif/file_utf32.txt')
if file_content is not None:
    print(file_content)

Successfully read with encoding: UTF-32
EnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarc